In [ ]:
from google.colab import drive
drive.mount('/content/drive')
debug = False

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.chdir("/content/drive/MyDrive/thesis/2_MODELING")

In [ ]:
from dotenv import load_dotenv

load_dotenv("../.env.prod")
import os
import pandas as pd
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel




In [ ]:
import torch


In [ ]:
model_name=os.getenv("model_name")
model_name

'sentence-transformers/all-mpnet-base-v2'

In [ ]:
train = pd.read_parquet("../files/processed/with_remote/train.parquet")


In [ ]:
print(len(train))

1391180


In [ ]:
train = train[train["label"]==1]

In [ ]:
print(len(train))

303401


In [ ]:
val = pd.read_parquet("../files/processed/with_remote/val.parquet")


In [ ]:
if debug:
  train = train.loc[:10000]
  val = val.loc[:10000]

In [ ]:
df_train=train[['candidate_id', 'vacant_id',"label", 'vacant_fourier_feature',  'candidate_fourier_features',  'job_chunks_input_ids', 'job_chunks_attention_mask', 'cand_chunks_input_ids','cand_chunks_attention_mask', 'vacant_remote']]
df_val=val[['candidate_id', 'vacant_id',"label", 'vacant_fourier_feature',  'candidate_fourier_features',  'job_chunks_input_ids', 'job_chunks_attention_mask', 'cand_chunks_input_ids','cand_chunks_attention_mask', 'vacant_remote']]


In [ ]:
import ast

def to_ll_int_col(col):
    def fix(x):
        if x is None:
            return []
        if isinstance(x, str):
            x = ast.literal_eval(x)
        return [[int(t) for t in seq] for seq in x]
    return col.apply(fix)

def to_ll_mask_col(col):
    def fix(x):
        if x is None:
            return []
        if isinstance(x, str):
            x = ast.literal_eval(x)
        return [[int(bool(t)) for t in seq] for seq in x]
    return col.apply(fix)

def to_1d_float_col(col):
    def fix(x):
        if x is None:
            return []
        if isinstance(x, str):
            x = ast.literal_eval(x)
        return [float(t) for t in x]
    return col.apply(fix)

In [ ]:



df_all = pd.concat([df_train, df_val], ignore_index=True)

cand_features = (
    df_all[[
        "candidate_id",
        "candidate_fourier_features",
        "cand_chunks_input_ids",
        "cand_chunks_attention_mask"
    ]]
    .drop_duplicates(subset=["candidate_id"])
    .set_index("candidate_id")
)

vac_features = (
    df_all[[
        "vacant_id",
        "vacant_fourier_feature",
        "job_chunks_input_ids",
        "job_chunks_attention_mask"
    ]]
    .drop_duplicates(subset=["vacant_id"])
    .set_index("vacant_id")
)

cand_features["cand_chunks_input_ids"]      = to_ll_int_col(cand_features["cand_chunks_input_ids"])
cand_features["cand_chunks_attention_mask"] = to_ll_mask_col(cand_features["cand_chunks_attention_mask"])
cand_features["candidate_fourier_features"] = to_1d_float_col(cand_features["candidate_fourier_features"])

vac_features["job_chunks_input_ids"]        = to_ll_int_col(vac_features["job_chunks_input_ids"])
vac_features["job_chunks_attention_mask"]   = to_ll_mask_col(vac_features["job_chunks_attention_mask"])
vac_features["vacant_fourier_feature"]      = to_1d_float_col(vac_features["vacant_fourier_feature"])

for df in [df_train, df_val]:
    df["cand_chunks_input_ids"]      = df["candidate_id"].map(cand_features["cand_chunks_input_ids"])
    df["cand_chunks_attention_mask"] = df["candidate_id"].map(cand_features["cand_chunks_attention_mask"])
    df["candidate_fourier_features"] = df["candidate_id"].map(cand_features["candidate_fourier_features"])

    df["job_chunks_input_ids"]       = df["vacant_id"].map(vac_features["job_chunks_input_ids"])
    df["job_chunks_attention_mask"]  = df["vacant_id"].map(vac_features["job_chunks_attention_mask"])
    df["vacant_fourier_feature"]     = df["vacant_id"].map(vac_features["vacant_fourier_feature"])



/tmp/ipython-input-330708063.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cand_chunks_input_ids"]      = df["candidate_id"].map(cand_features["cand_chunks_input_ids"])
/tmp/ipython-input-330708063.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cand_chunks_attention_mask"] = df["candidate_id"].map(cand_features["cand_chunks_attention_mask"])
/tmp/ipython-input-330708063.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [ ]:
from model.loader import make_collate_fn_chunks, PairDatasetPNChunked
from model.siames import SiameseTwoTower

In [ ]:

# 1) Dataset + collate
tmp_tok = AutoTokenizer.from_pretrained(os.getenv("model_name"), use_fast=True)

pad_id = tmp_tok.pad_token_id
print(pad_id)

collate_fn = make_collate_fn_chunks(pad_id=0)

train_ds = PairDatasetPNChunked(df_train)

train_loader = DataLoader(train_ds,
                          batch_size=16,
                          shuffle=True,
                          collate_fn=collate_fn,
                          num_workers=0)  # <-- set to 0


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1


In [ ]:
val_ds = PairDatasetPNChunked(df_val)

val_loader = DataLoader(val_ds,
                          batch_size=2,
                          shuffle=True,
                          collate_fn=collate_fn,
                          num_workers=0)  # <-- set to 0

In [ ]:
pos = df_train["label"].sum()
neg = len(df_train) - pos
pos_weight = neg / pos
pos_weight

np.float64(0.0)

In [ ]:
pos = df_val["label"].sum()
neg = len(df_val) - pos
pos_weight = neg / pos
pos_weight

np.float64(4.377979334690407)

In [ ]:
len(df_train)

303401

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [ ]:
job_loc_in=len(df_train.iloc[0]["vacant_fourier_feature"])
cand_loc_in=len(df_train.iloc[0]["candidate_fourier_features"])

In [ ]:
torch.cuda.empty_cache()

In [ ]:

# 2) Model (late fusion on vacancy & candidate if you want both)
model = SiameseTwoTower(
    encoder_name=model_name,
    proj_dim=256,
    job_loc_in=job_loc_in,
    cand_loc_in=cand_loc_in,
    max_length=256).to(device)



config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler

from model.train import train_one_epoch, evaluate


def train_and_validate(
    model,
    train_loader: DataLoader,
    valid_loader: DataLoader = None,
    num_epochs: int = 5,
    lr: float = 2e-5,
    weight_decay: float = 0.01,
    grad_clip: float = 1.0,
    grad_accum_steps: int = 1,
    pos_weight: float = None,
    save_path: str = "twotower_best.pt",
):
    """
    Entrena y valida un modelo TwoTower en GPU (si hay), y guarda el mejor checkpoint.
    Las dos torres usan el mismo tipo de input (texto chunked + features de localización).
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Usando device:", device)
    model.to(device)

    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr,
        weight_decay=weight_decay,
    )

    # <-- aquí activamos AMP si hay CUDA (A100 en tu caso)
    scaler = GradScaler() if device.type == "cuda" else None

    best_metric = -float("inf")
    best_state = None

    for epoch in range(1, num_epochs + 1):
        # ---- TRAIN ----
        train_metrics = train_one_epoch(
            model=model,
            loader=train_loader,
            optimizer=optimizer,
            device=device,
            pos_weight=pos_weight,
            grad_clip=grad_clip,
            grad_accum_steps=grad_accum_steps,
            scaler=scaler,            
            loss_type="info_nce"

        )

        val_metrics = evaluate(model=model, loader=valid_loader, device=device)

        metric = val_metrics["roc_auc"] 
        print(
            f"[Epoch {epoch:03d}] "
            f"train_loss={train_metrics['loss']:.4f} | "
            f"val_loss={val_metrics['loss']:.4f} | "
            f"val_acc={val_metrics['accuracy']:.4f} | "
            f"val_auc={val_metrics['roc_auc']:.4f}"
        )

        if metric > best_metric and not debug:
            best_metric = metric
            best_state = {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "train_loss": train_metrics["loss"],
                "val_metrics": val_metrics,
                "scaler_state_dict": scaler.state_dict() if scaler is not None else None,
            }
            torch.save(best_state, save_path)
            print(f"  -> Nuevo mejor modelo guardado en: {save_path}")

    return model, best_state


In [ ]:
model, best_state = train_and_validate(
    model=model,
    train_loader=train_loader,
    valid_loader=val_loader,
    num_epochs=10,
    grad_accum_steps=1,
    lr=2e-5,
    save_path="./twotower_best_INFO_NCE_2.pt",
    pos_weight = pos_weight
)

Usando device: cuda
Total steps this epoch: 18963


/tmp/ipython-input-3073503756.py:35: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if device.type == "cuda" else None
/content/drive/MyDrive/thesis/2_MODELING/model/train.py:59: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Step 100/18963 (0.5%), batch_loss=2.1221
Step 200/18963 (1.1%), batch_loss=2.5089
Step 300/18963 (1.6%), batch_loss=1.9238
Step 400/18963 (2.1%), batch_loss=1.3505
Step 500/18963 (2.6%), batch_loss=2.1613
Step 600/18963 (3.2%), batch_loss=1.5834
Step 700/18963 (3.7%), batch_loss=2.3370
Step 800/18963 (4.2%), batch_loss=1.7541
Step 900/18963 (4.7%), batch_loss=2.5438
Step 1000/18963 (5.3%), batch_loss=1.4897
Step 1100/18963 (5.8%), batch_loss=1.9421
Step 1200/18963 (6.3%), batch_loss=2.3224
Step 1300/18963 (6.9%), batch_loss=2.2095
Step 1400/18963 (7.4%), batch_loss=2.0835
Step 1500/18963 (7.9%), batch_loss=1.7070
Step 1600/18963 (8.4%), batch_loss=2.3848
Step 1700/18963 (9.0%), batch_loss=2.4047
Step 1800/18963 (9.5%), batch_loss=1.4062
Step 1900/18963 (10.0%), batch_loss=1.9831
Step 2000/18963 (10.5%), batch_loss=1.7371
Step 2100/18963 (11.1%), batch_loss=2.1392
Step 2200/18963 (11.6%), batch_loss=1.9178
Step 2300/18963 (12.1%), batch_loss=1.7157
Step 2400/18963 (12.7%), batch_loss=1.

/content/drive/MyDrive/thesis/2_MODELING/model/train.py:147: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


[Epoch 001] train_loss=1.3122 | val_loss=7.2068 | val_acc=0.1869 | val_auc=0.5402
  -> Nuevo mejor modelo guardado en: ./twotower_best_INFO_NCE_2.pt
Total steps this epoch: 18963


...

Step 100/18963 (0.5%), batch_loss=1.1326
Step 200/18963 (1.1%), batch_loss=1.0354
Step 300/18963 (1.6%), batch_loss=1.6043
Step 400/18963 (2.1%), batch_loss=1.0736
Step 500/18963 (2.6%), batch_loss=1.1896
Step 600/18963 (3.2%), batch_loss=1.4255
Step 700/18963 (3.7%), batch_loss=1.0589
Step 800/18963 (4.2%), batch_loss=0.7580
Step 900/18963 (4.7%), batch_loss=0.7911
Step 1000/18963 (5.3%), batch_loss=1.1530
Step 1100/18963 (5.8%), batch_loss=1.4593
Step 1200/18963 (6.3%), batch_loss=0.6771
Step 1300/18963 (6.9%), batch_loss=1.2578
Step 1400/18963 (7.4%), batch_loss=0.9420
Step 1500/18963 (7.9%), batch_loss=1.0669
Step 1600/18963 (8.4%), batch_loss=0.8593
Step 1700/18963 (9.0%), batch_loss=0.7960
Step 1800/18963 (9.5%), batch_loss=0.6303
Step 1900/18963 (10.0%), batch_loss=1.0688
Step 2000/18963 (10.5%), batch_loss=1.5831
Step 2100/18963 (11.1%), batch_loss=1.1561
Step 2200/18963 (11.6%), batch_loss=1.5200
Step 2300/18963 (12.1%), batch_loss=0.6461
Step 2400/18963 (12.7%), batch_loss=0.

/content/drive/MyDrive/thesis/2_MODELING/model/train.py:147: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


[Epoch 002] train_loss=1.0175 | val_loss=6.6900 | val_acc=0.1879 | val_auc=0.5445
  -> Nuevo mejor modelo guardado en: ./twotower_best_INFO_NCE_2.pt
Total steps this epoch: 18963


/content/drive/MyDrive/thesis/2_MODELING/model/train.py:59: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Step 100/18963 (0.5%), batch_loss=1.0846
Step 200/18963 (1.1%), batch_loss=0.9200
Step 300/18963 (1.6%), batch_loss=0.8631
Step 400/18963 (2.1%), batch_loss=1.2582
Step 500/18963 (2.6%), batch_loss=0.5726
Step 600/18963 (3.2%), batch_loss=0.8314
Step 700/18963 (3.7%), batch_loss=1.1314
Step 800/18963 (4.2%), batch_loss=0.6835
Step 900/18963 (4.7%), batch_loss=1.4773
Step 1000/18963 (5.3%), batch_loss=1.5648
Step 1100/18963 (5.8%), batch_loss=0.9631
Step 1200/18963 (6.3%), batch_loss=0.9058
Step 1300/18963 (6.9%), batch_loss=1.1184
Step 1400/18963 (7.4%), batch_loss=1.2198
Step 1500/18963 (7.9%), batch_loss=0.9900
Step 1600/18963 (8.4%), batch_loss=0.8127
Step 1700/18963 (9.0%), batch_loss=1.7653
Step 1800/18963 (9.5%), batch_loss=1.6691
Step 1900/18963 (10.0%), batch_loss=0.5788
Step 2000/18963 (10.5%), batch_loss=0.8608
Step 2100/18963 (11.1%), batch_loss=1.0163
Step 2200/18963 (11.6%), batch_loss=0.8977
Step 2300/18963 (12.1%), batch_loss=0.5730
Step 2400/18963 (12.7%), batch_loss=1.